In [5]:
import pandas as pd
import numpy as np
import sys
import pickle
import warnings
warnings.filterwarnings("ignore")

sys.path.insert(0, '..')
import DataModule
from DataModule.Data_Preparation import CoronnaCERTAINDataset
import EvaluationModule

import xgboost as xgb
import sklearn
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import ShuffleSplit,cross_val_score
from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn.pipeline import Pipeline

import submitit

In [ ]:
    remove_low_DAS_list = [True, False]
#     remove_low_DAS_list = [True]
    challenge_list = ["regression", "regression_delta", "classification", "binary_classification"]
#     challenge_list = ["classification", "binary_classification"]
    imputation_list = ["SimpleFill", "KNN", "SoftImpute", "IterativeImputer"]
#     imputation_list = ["KNN"]
    process_approach_list = ['KVB','SC']
#     process_approach_list = ['SC']
    patient_group_list = ["all", "bioexp nTNF", "bionaive TNF", "bionaive orencia", "KVB"]
#     patient_group_list = ["bionaive TNF"]
    for remove_low_DAS in remove_low_DAS_list:
        for challenge in challenge_list:
            for imputation in imputation_list:
                for process_approach in process_approach_list:
                    for patient_group in patient_group_list:

In [6]:
def train(challenge, process_approach, imputation, patient_group, remove_low_DAS):
    dataset = CoronnaCERTAINDataset(
        library_root = '/Users/gaskell/Dropbox/Mac/Desktop/Autoimmune_Disease/Code/ML_RA_EHR/Dataset/',
        challenge = challenge, #option: regression, regression_delta, classification, binary_classification
        dataset = 'CORRONA CERTAIN', 
        process_approach = process_approach, #option: KVB, SC
        imputation = imputation, #option: SimpleFill, KNN, SoftImpute, BiScaler, NuclearNormMinimization, IterativeImputer, IterativeSVD, None(raw)
        patient_group =patient_group, #option: "all", "bioexp nTNF", "bionaive TNF", "bionaive orencia", "KVB"
        drug_group = 'all', #option: "all", "actemra", "cimzia", "enbrel", "humira", "orencia", "remicade", "rituxan", "simponi"
        time_points = (0,3), 
        train_test_rate = 0.8,
        remove_low_DAS = remove_low_DAS,
        save_csv = False, 
        random_state = 2022)
    # read train, test from dataloader
    train_set, train_loc = dataset.get_train()
    test_set, test_loc = dataset.get_test()

    aml = EvaluationModule.AutoBuild(seed=dataset.random_state, project_name="SC_Jul8_2", challenge=dataset.challenge)
    # define models
    if "regression" in dataset.challenge:
        model = RandomForestRegressor()
    elif "classification" in dataset.challenge:
        model = RandomForestClassifier()
    aml.validate("rf", model, train_set, test_set)
    aml.validation_output(dataset)
    aml.test_output(dataset)

In [7]:
# executor is the submission interface (logs are dumped in the folder)
executor = submitit.AutoExecutor(folder="log_test")
# set timeout in min, and partition for running the job
executor.update_parameters(timeout_min=60, slurm_partition="dev")
job = executor.submit(train, True, "classification", "SC", "SoftImpute", "bionaive TNF")  # 
print(job.job_id)  # ID of your job

output = job.result()  # waits for completion and returns output
# assert output == 12  # 5 + 7 = 12...  your addition was computed in the cluster

14251


FailedJobError: Job (task=0) failed during processing with trace:
----------------------
Traceback (most recent call last):
  File "/Users/gaskell/miniforge_x86_64/envs/ADPred_new/lib/python3.10/site-packages/submitit/core/submission.py", line 50, in process_job
    delayed = utils.DelayedSubmission.load(paths.submitted_pickle)
  File "/Users/gaskell/miniforge_x86_64/envs/ADPred_new/lib/python3.10/site-packages/submitit/core/utils.py", line 142, in load
    obj = pickle_load(filepath)
  File "/Users/gaskell/miniforge_x86_64/envs/ADPred_new/lib/python3.10/site-packages/submitit/core/utils.py", line 233, in pickle_load
    return pickle.load(ifile)
ModuleNotFoundError: No module named 'DataModule'

----------------------
You can check full logs with 'job.stderr(0)' and 'job.stdout(0)'or at paths:
  - /Users/gaskell/Dropbox/Mac/Desktop/Autoimmune_Disease/Code/ML_RA_EHR/ModelModule/log_test/14251_0_log.err
  - /Users/gaskell/Dropbox/Mac/Desktop/Autoimmune_Disease/Code/ML_RA_EHR/ModelModule